<a href="https://colab.research.google.com/github/pdrobny/MonReader/blob/main/P4_tts_sesame_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install --upgrade transformers

In [1]:
import IPython.display as ipd
from google.colab import userdata
import numpy as np
import pandas as pd
import os
import PIL.Image
import torch
import torchaudio
from transformers import CsmForConditionalGeneration, AutoProcessor


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.filterwarnings('ignore')
print("Imported libraries.")

Imported libraries.


In [2]:
# load files from google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Sesame TTS

In [3]:
# prompt: hugging face login

from huggingface_hub import notebook_login
notebook_login()

In [4]:
# Intialize tts model dia
model_id = "sesame/csm-1b"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
 #load the model and the processor
processor = AutoProcessor.from_pretrained(model_id)
model = CsmForConditionalGeneration.from_pretrained(model_id, device_map=device)

## Test text

In [6]:
# prepare the inputs
text = "[0]Hello from Sesame." # `[0]` for speaker id 0
inputs = processor(text, add_special_tokens=True).to(device)

# infer the model
audio = model.generate(**inputs, output_audio=True, max_new_tokens=1024)
processor.save_audio(audio, "example.wav")

In [ ]:
ipd.Audio("example.wav")

## Reading first paragraph from The Theory of Everything - extracted from Gemini

In [ ]:
# Read text from the file
text_file_path = "/content/drive/MyDrive/Apziva/MonReader/extract/p3_extract.txt"
try:
    with open(text_file_path, 'r') as f:
        book_page_text = f.read().strip()
except FileNotFoundError:
    print(f"Error: Text file not found at {text_file_path}")
    book_page_text = "" # Set empty string if file not found

if book_page_text:
    # Remove newline characters
    cleaned_text = book_page_text.replace('\n', ' ')

    # Format the text for Sesame (assuming speaker id 0)
    formatted_text = f"[0]{cleaned_text}"
    print(formatted_text)

    # prepare the inputs
    inputs = processor(formatted_text, add_special_tokens=True).to(device)

    # infer the model
    audio_from_file = model.generate(**inputs, output_audio=True, max_new_tokens=1024)

    # Save the audio
    output_audio_path = "p3_good_audio_full.wav"
    processor.save_audio(audio_from_file, output_audio_path)
    print(f"Audio saved to {output_audio_path}")

    # Play the audio
    display(ipd.Audio(output_audio_path))
else:
    print("No text found in the file to generate audio.")